# First Task

In [1]:
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [2]:
def count_parameters(mod):
    return np.sum(list(np.prod(l.cpu().detach().numpy().shape) for l in mod.parameters()))

In [3]:
def print_params(files):
    for name, file in files.items():
        params = np.sum([count_parameters(torch.load("./models/best_model_" + f))
                       for f in file])
        print(name, params, sep=": ")

In [4]:
class Custom_RNN(torch.nn.Module):

    def __init__(self, hiddens, recur,
                 regressors,
                 p_dropout=0.3):
        self.recur_input, self.recur_output = recur
        # recur_input in [0, len(hiddens))
        # recur_output in [-1, len(hiddens)]
        assert 0 <= self.recur_input <= len(hiddens)
        assert -1 <= self.recur_output < len(hiddens)
        assert len(regressors) == 2

        super(Custom_RNN, self).__init__()
        # input_shape = len(FEATURES)  # new task
        input_shape = len(FEATURES) + len(TARGET)  # old task
        self.previous_size = hiddens[self.recur_output] \
            if self.recur_output >= 0 else len(TARGET)
        
        hiddens_in = [input_shape] + hiddens

        hiddens_in[self.recur_input] += self.previous_size

        self.dropout = torch.nn.Dropout(p=p_dropout)
        self.layers = torch.nn.ModuleList(
            [torch.nn.Linear(in_shape, out_shape) \
             for in_shape, out_shape in zip(hiddens_in, hiddens)])
        self.layers_fn = [torch.nn.functional.relu \
                          for _ in self.layers]

        regressor_list = []
        for features, regressor in zip([1, 3], regressors):
            if type(regressor) != list:
                regressor = [regressor]
            input_shape = [hiddens[-1]] + regressor
            y_net = [torch.nn.ReLU() for _ in input_shape + regressor]
            y_net[::2] = [torch.nn.Linear(in_shape, out_shape)
                          for in_shape, out_shape
                          in zip(input_shape, regressor + [features])]
            regressor_list.append(torch.nn.Sequential(*y_net))
        self.regressors = torch.nn.ModuleList(regressor_list)

    def initialize_prev(self, x):
        return torch.zeros(len(x), self.previous_size).to(DEVICE)

    def forward(self, data, lag=None):
        _, l, _ = data.shape
        x = data[:, 0, :]  # .reshape(-1, len(FEATURES))
        if lag is None:  # first recursion
            lag = self.initialize_prev(x)
        for i, (layer, fn) in enumerate(zip(self.layers, self.layers_fn)):
            if i == self.recur_input:
                x = torch.cat([x, lag], axis=1)
            x = fn(layer(x))
            x = self.dropout(x)
            if i == self.recur_output and l > 1:
                return self.forward(data[:, 1:, :], x)
        x = torch.cat([regressor(x) for regressor in self.regressors],
                      axis=1)
        if self.recur_output == -1 and l > 1:
            return self.forward(data[:, 1:, :], x)
        return x

In [5]:
class CNN_Net_2(nn.Module):  # AML_CNN, cella 11
    def __init__(self, batch, in_c, out, 
                 filtro1, filtro2,
                 num_ser, 
                 neuroni1, neuroni2,
                 kernel1, kernel2, kernel3, kernel4, 
                 #padding1, padding2, padding3, padding4, 
                 stride1, stride2, stride3, stride4):
        super(CNN_Net_2, self).__init__()
        self.batch_size = 1
        self.in_c = in_c
        l0 = num_ser
        l1 = outputSize(l0,kernel1,stride1,0)
        l2 = outputSize(l1,kernel2,stride2,0)
        l3 = outputSize(l2,kernel3,stride3,0)
        l4 = outputSize(l3,kernel4,stride4,0)
        #print(l4)

        self.conv1 = nn.Conv1d(in_channels= in_c, 
                               out_channels= filtro1, 
                               kernel_size= kernel1,
                               stride= stride1,
                               padding=0)    
        self.pool1 = nn.MaxPool1d(kernel2,stride=stride2,padding=0) 
        self.conv2 = nn.Conv1d(filtro1,filtro2,kernel3,stride=stride3,padding=0)
        self.pool2 = nn.MaxPool1d(kernel4,stride=stride4,padding=0)
        self.fc1 = nn.Linear(filtro2*l4, neuroni1)
        self.fc2 = nn.Linear(neuroni1, neuroni2)
        self.fc3 = nn.Linear(neuroni2, out)

    def forward(self, x):
        batch_size, _, _ = x.shape
        x =  self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(batch_size, self.num_flat_features(x))
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x))
        return  self.fc3(x)

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:       # Get the products
            num_features *= s
        return num_features

In [6]:
class GRU_Net(nn.Module):  # AML_GRU, cella 7
    def __init__(self, features, hidden, h1, h2, out):
        super(GRU_Net, self).__init__()
        self.h1 = h1
        self.h2 = h2

        self.hidden_size = hidden
        self.features = features

        self.gru = nn.GRU(self.features, self.hidden_size, 1, #feature_size, hidden_size, num_layer
                            batch_first = True) 
        self.fc1 = nn.Linear(self.hidden_size, 
                                h1)
        self.out = nn.Linear(h1, out)

        if self.h2 != 0:
            self.fc2 = nn.Linear(h1, h2)
            self.out = nn.Linear(h2, out)
        
        #we do it stateless so there is no need for the hidden_state
        #self.hidden = None #torch.randn(1, ??, self.hidden) #num_layer, batch, hidden_size

    def forward(self, x):
        batch_size, _, _ = x.shape 
        
        x, _ =  self.gru(x)
        x = F.tanh(x[:,-1].view(batch_size, -1))
        
        x = F.relu(self.fc1(x))
        if self.h2 != 0:
            x = F.relu(self.fc2(x))
        return self.out(x)

In [7]:
class LSTM_Net(nn.Module):  # AML_LSTM, cella 9
    def __init__(self, features, hidden, h1, h2, out):
        super(LSTM_Net, self).__init__()
        self.h1 = h1
        self.h2 = h2

        self.hidden_size = hidden
        self.features = features

        self.lstm = nn.LSTM(self.features, self.hidden_size, 1, #feature_size, hidden_size, num_layer
                            batch_first = True) 
        self.fc1 = nn.Linear(self.hidden_size, 
                                h1)
        self.out = nn.Linear(h1, out)

        if self.h2 != 0:
            self.fc2 = nn.Linear(h1, h2)
            self.out = nn.Linear(h2, out)
        
        #we do it stateless so there is no need for the hidden_state
        #self.hidden = None #torch.randn(1, ??, self.hidden) #num_layer, batch, hidden_size

    def forward(self, x):
        batch_size, _, _ = x.shape 
        
        x, _ =  self.lstm(x)
        x = F.tanh(x[:,-1].view(batch_size, -1))
        
        x = F.relu(self.fc1(x))
        if self.h2 != 0:
            x = F.relu(self.fc2(x))
        return self.out(x)

## MSE

In [8]:
mse_1 = {
    "RNN": ["rnn_MSE"],
    "CNN": ["cnn", "cnn_stator"],
    "GRU": ["gru", "gru_stator"],
    "LSTM": ["lstm", "lstm_stator"]
}
print_params(mse_1)

RNN: 2485
CNN: 777613
GRU: 24519
LSTM: 62882


/home/fede/.anaconda/lib/python3.7/site-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.GRU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/fede/.anaconda/lib/python3.7/site-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


## PK Loss

In [9]:
custom_loss_1 = {
    "RNN": ["rnn_CustomLoss"],
    "CNN": ["cnn_newLoss"],
    "GRU": ["gru_new_loss"],
    "LSTM": ["lstm_new_loss"]
}
print_params(custom_loss_1)

RNN: 3190
CNN: 11840
GRU: 7369
LSTM: 97249


# Second Task

In [10]:
class Reg_CNN_Net(nn.Module):  # SecondTaskCNN_LSTM, cella 6
    def __init__(self, features, seq_len, 
                 conv1, conv2, kernel1, kernel2,
                 h1, h2, out):
        super(Reg_CNN_Net, self).__init__()
        #self.h1 = h1
        self.h2 = h2
        #self.conv1 = conv1
        self.conv2 = conv2

        self.features = features
        self.seq_len = seq_len
        
        self.c1 = nn.Conv1d(self.seq_len, conv1, kernel1)
        
        h0 = outputSize(self.features, kernel1, 1, 0)*conv1
        if conv2 != 0:
            self.c2 = nn.Conv1d(conv1, conv2, kernel2)
        
        
            h0 = outputSize(outputSize(self.features, kernel1, 1, 0), 
                            kernel2, 1 ,0)*conv2

        self.fc1 = nn.Linear(h0, h1)
        

        if self.h2 != 0:
            self.fc2 = nn.Linear(h1, h2)
            
            self.out = nn.Linear(h2, out)
        else:
            self.out = nn.Linear(h1, out)

    def forward(self, x):
        batch_size, _, _ = x.shape 
        
        x1 = F.relu(self.c1(x))
        if self.conv2!=0:
            x = F.relu(self.c2(x1))
        else:
            x = x1
        
        x = x.view(batch_size,-1)
        
        x = F.relu(self.fc1(x))
        if self.h2 != 0:
            x = F.relu(self.fc2(x))
        return self.out(x)

In [11]:
class LSTM_Net(nn.Module):  # SecondTaskCNN_LSTM, cella 6
    def __init__(self, features, hidden, h1, h2, out):
        super(LSTM_Net, self).__init__()
        self.h1 = h1
        self.h2 = h2

        self.hidden_size = hidden
        self.features = features

        self.lstm = nn.LSTM(self.features, self.hidden_size, 1, #feature_size, hidden_size, num_layer
                            batch_first = True) 
        self.fc1 = nn.Linear(self.hidden_size, 
                                h1)
        self.out = nn.Linear(h1, out)

        if self.h2 != 0:
            self.fc2 = nn.Linear(h1, h2)
            self.out = nn.Linear(h2, out)
        
        #we do it stateless so there is no need for the hidden_state
        #self.hidden = None #torch.randn(1, ??, self.hidden) #num_layer, batch, hidden_size

    def forward(self, x):
        batch_size, _, _ = x.shape 
        
        x, _ =  self.lstm(x)
        x = F.tanh(x[:,-1].view(batch_size, -1))
        
        x = F.relu(self.fc1(x))
        if self.h2 != 0:
            x = F.relu(self.fc2(x))
        return self.out(x)

In [12]:
class Custom_RNN(torch.nn.Module):  # Rete2, cella 19

    def __init__(self, hiddens, recur,
                 regressors,
                 p_dropout=0.3):
        self.recur_input, self.recur_output = recur
        # recur_input in [0, len(hiddens))
        # recur_output in [-1, len(hiddens)]
        assert 0 <= self.recur_input <= len(hiddens)
        assert -1 <= self.recur_output < len(hiddens)
        assert len(regressors) == 2

        super(Custom_RNN, self).__init__()
        input_shape = len(FEATURES)  # new task
        # input_shape = len(FEATURES) + len(TARGET)  # old task
        self.previous_size = hiddens[self.recur_output] \
            if self.recur_output >= 0 else len(TARGET)
        
        hiddens_in = [input_shape] + hiddens

        hiddens_in[self.recur_input] += self.previous_size

        self.dropout = torch.nn.Dropout(p=p_dropout)
        self.layers = torch.nn.ModuleList(
            [torch.nn.Linear(in_shape, out_shape) \
             for in_shape, out_shape in zip(hiddens_in, hiddens)])
        self.layers_fn = [torch.nn.functional.relu \
                          for _ in self.layers]

        regressor_list = []
        for features, regressor in zip([1, 3], regressors):
            if type(regressor) != list:
                regressor = [regressor]
            input_shape = [hiddens[-1]] + regressor
            y_net = [torch.nn.ReLU() for _ in input_shape + regressor]
            y_net[::2] = [torch.nn.Linear(in_shape, out_shape)
                          for in_shape, out_shape
                          in zip(input_shape, regressor + [features])]
            regressor_list.append(torch.nn.Sequential(*y_net))
        self.regressors = torch.nn.ModuleList(regressor_list)

    def initialize_prev(self, x):
        return torch.zeros(len(x), self.previous_size).to(DEVICE)

    def forward(self, data, lag=None):
        _, l, _ = data.shape
        x = data[:, 0, :]  # .reshape(-1, len(FEATURES))
        if lag is None:  # first recursion
            lag = self.initialize_prev(x)
        for i, (layer, fn) in enumerate(zip(self.layers, self.layers_fn)):
            if i == self.recur_input:
                x = torch.cat([x, lag], axis=1)
            x = fn(layer(x))
            x = self.dropout(x)
            if i == self.recur_output and l > 1:
                return self.forward(data[:, 1:, :], x)
        x = torch.cat([regressor(x) for regressor in self.regressors],
                      axis=1)
        if self.recur_output == -1 and l > 1:
            return self.forward(data[:, 1:, :], x)
        return x

## MSE

In [13]:
mse_2 = {
    "RNN": ["rnn2_MSE"],
    "CNN": [],
    "GRU": [],
    "LSTM": []
}
print_params(mse_2)

RNN: 1740
CNN: 0.0
GRU: 0.0
LSTM: 0.0


## PK Loss

In [14]:
custom_loss_2 = {
    "RNN": ["rnn2_Custom"],
    "CNN": [],
    "GRU": [],
    "LSTM": []
}
print_params(custom_loss_2)

RNN: 1740
CNN: 0.0
GRU: 0.0
LSTM: 0.0
